In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# user-model import
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my_playground/models')

In [3]:
from google.colab import files # 데이터 불러오기
file_uploaded=files.upload()   # chap06/data/dogs-vs-cats.zip 데이터 불러오기

Saving dogs-vs-cats.zip to dogs-vs-cats.zip


In [3]:
!unzip dogs-vs-cats.zip -d dogs-vs-cats/  #dogs-vs-cats 폴더 만들어 압축 풀기

Archive:  dogs-vs-cats.zip
replace dogs-vs-cats/Cat/cat.0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Library

In [3]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torch.autograd import Variable
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import cv2
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import random
from matplotlib import pyplot as plt
import pandas as pd
import LeNet5, AlexNet, VGGNet, ResNet
from collections import namedtuple

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }

    def __call__(self, img, mode):
        if mode == "test": #test 도 val과 같이 처리
            mode = "val"
        return self.data_transform[mode](img)

In [6]:
class DogvsCatDataset(Dataset):
    def __init__(self, file_list, transform=None, mode='train'):
        self.file_list = file_list
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.mode)
        label = img_path.split('/')[-1].split('.')[0]
        if self.mode == "test":
            return img_transformed
        else:
            if label == 'dog':
                label = 1
            elif label == 'cat':
                label = 0
            return img_transformed, label

In [8]:
class Trainer:
    def __init__(self, model_name = "LeNet5", size=224):
        self.cat_path = "dogs-vs-cats/Cat/"
        self.dog_path = "dogs-vs-cats/Dog/"
        self.model_name = model_name
        self.save_path = f"./best_{model_name}_model.pth"
        self.vgg11_config = [64,'M',128,'M',256,256,'M',512,512,'M',512,512,'M']


        ResNetConfig = namedtuple("ResNetConfig", ['block','n_blocks','channels'])
        self.resnet50_config = ResNetConfig(block=ResNet.Bottleneck,
                                    n_blocks =[3,4,6,3],
                                    channels = [64,128,256,512])



        self.size = size
        self.mean = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

        self.BATCH_SIZE = 32
        self.NUM_WORKERS = 0
        self.EPOCH = 10
        self.DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.SEED = 42
        seed_everything(self.SEED)

    def setup(self):
        """ setup """
        cat_images_filepaths = sorted([os.path.join(self.cat_path, f) for f in os.listdir(self.cat_path)])
        dog_images_filepaths = sorted([os.path.join(self.dog_path, f) for f in os.listdir(self.dog_path)])
        images_filepaths = [*cat_images_filepaths, *dog_images_filepaths]
        correct_images_filepaths = [i for i in images_filepaths if cv2.imread(i) is not None]
        random.shuffle(correct_images_filepaths)

        train_images_filepaths = correct_images_filepaths[:400]
        val_images_filepaths = correct_images_filepaths[400:-10]
        test_images_filepaths = correct_images_filepaths[-10:]

        train_dataset = DogvsCatDataset(train_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='train')
        val_dataset = DogvsCatDataset(val_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='val')
        test_dataset = DogvsCatDataset(test_images_filepaths, transform=ImageTransform(self.size, self.mean, self.std), mode='test')

        self.train_dataloader = DataLoader(
            dataset=train_dataset,
            batch_size=self.BATCH_SIZE,
            shuffle=True,
            num_workers=self.NUM_WORKERS,
            pin_memory=True
        )
        self.val_dataloader = DataLoader(
                    dataset=val_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )
        self.test_dataloader = DataLoader(
                    dataset=test_dataset,
                    batch_size=self.BATCH_SIZE,
                    shuffle=False,
                )

        self.model_dict = {"LeNet5": LeNet5.LeNet_5(),
                           "AlexNet": AlexNet.AlexNet(),
                           "VGGNet": VGGNet.VGGNet(self.vgg11_config),
                           "ResNet": ResNet.ResNet(self.resnet50_config, output_dim=2)} # 점차 쌓아 나감.
        self.model = self.model_dict[self.model_name]
        self.loss_fn = nn.CrossEntropyLoss() #criterion
        self.optimizer = optim.SGD(self.model.parameters(), lr=0.001, momentum=0.9)

    def train(self):
        self.model.to(self.DEVICE)
        self.loss_fn.to(self.DEVICE)
        best_acc = 0.0

        for epoch in range(self.EPOCH):
            self.model.train()
            print('Epoch {}/{}'.format(epoch + 1, self.EPOCH))
            print('-'*20)
            epoch_loss = 0.0
            epoch_corrects = 0

            for inputs, labels in tqdm(self.train_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                loss.backward()
                self.optimizer.step()

                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)

            epoch_loss = epoch_loss / len(self.train_dataloader.dataset)
            epoch_acc = epoch_corrects.double() / len(self.train_dataloader.dataset)
            print('[Train] Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc)) #train은 필요 없을 것 같다.

            val_loss, val_acc = self.valid()
            print('[Val] Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

            if val_acc > best_acc:
                best_acc = val_acc
                best_model = self.model.state_dict()
                torch.save(self.model.state_dict(), self.save_path)

    def valid(self):
        self.model.eval()
        val_loss, val_corrects = 0.0, 0
        with torch.no_grad():
            for inputs, labels in tqdm(self.val_dataloader):
                inputs = inputs.to(self.DEVICE)
                labels = labels.to(self.DEVICE)

                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = self.loss_fn(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

            val_loss = val_loss / len(self.val_dataloader.dataset)
            val_acc = val_corrects.double() / len(self.val_dataloader.dataset)
        return val_loss, val_acc

    def test(self):
        model = self.model_dict[self.model_name].to(self.DEVICE)
        model.load_state_dict(torch.load(self.save_path))
        model.eval()
        id_list, pred_list = [] , []
        _id=0
        with torch.no_grad():
            for inputs in tqdm(self.test_dataloader):
                for img in inputs:
                    img = img.unsqueeze(0) #왜 test 만 unsqueeze? ->  input차원을 맞춰주기 위해서
                    img = img.to(self.DEVICE)

                    outputs = model(img)
                    preds = F.softmax(outputs, dim=1)[:, 1].tolist()
                    id_list.append(_id)
                    pred_list.append(preds[0])

        res = pd.DataFrame({
            'id': id_list,
            'label': pred_list
        })

        res.sort_values(by='id', inplace=True)
        res.reset_index(drop=True, inplace=True)

        res.to_csv(f'{self.model_name}.csv', index=False)
        return res


# LeNet5

In [17]:
lenet_trainer = Trainer(model_name = "LeNet5",size=224)
lenet_trainer.setup()

In [28]:
lenet_trainer.train()

Epoch 1/10
--------------------


<ipython-input-26-92bc4c47a319>:82: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

tensor([[0.4793, 0.5207],
        [0.4969, 0.5031],
        [0.5127, 0.4873],
        [0.4868, 0.5132],
        [0.4917, 0.5083],
        [0.5000, 0.5000],
        [0.5034, 0.4966],
        [0.5077, 0.4923],
        [0.5081, 0.4919],
        [0.4861, 0.5139],
        [0.5000, 0.5000],
        [0.4989, 0.5011],
        [0.5122, 0.4878],
        [0.5268, 0.4732],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.4838, 0.5162],
        [0.4805, 0.5195],
        [0.5000, 0.5000],
        [0.5098, 0.4902],
        [0.4978, 0.5022],
        [0.4880, 0.5120],
        [0.4717, 0.5283],
        [0.5000, 0.5000],
        [0.5000, 0.5000],
        [0.4899, 0.5101],
        [0.5162, 0.4838],
        [0.5013, 0.4987],
        [0.4826, 0.5174],
        [0.5098, 0.4902],
        [0.5075, 0.4925]], device='cuda:0', grad_fn=<SoftmaxBackward0>) tensor([1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 0], device='c

<ipython-input-26-92bc4c47a319>:116: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


  0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
lenet_trainer.test()

<ipython-input-10-4056cc541dd5>:125: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

,id,label
0,0,0.366190
1,0,0.552481
2,0,0.495040
3,0,0.442596
4,0,0.582851
5,0,0.578239
6,0,0.388221
7,0,0.480908
8,0,0.512693
9,0,0.481142


# AlexNet

In [ ]:
alex_trainer = Trainer(model_name = "AlexNet",size=227)
alex_trainer.setup()

In [ ]:
alex_trainer.train()

Epoch 1/10
--------------------


<ipython-input-19-b7ca38ab0b12>:71: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6932 Acc: 0.4875


<ipython-input-19-b7ca38ab0b12>:103: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.6925 Acc: 0.5109
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6929 Acc: 0.5100


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.6924 Acc: 0.5109


In [ ]:
alex_trainer.test()

<ipython-input-25-3a1ea5c2dccb>:125: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

,id,label
0,0,0.501077
1,0,0.501330
2,0,0.500585
3,0,0.502473
4,0,0.501585
5,0,0.500890
6,0,0.500899
7,0,0.501411
8,0,0.500958
9,0,0.500881


# VGGNet

In [ ]:
vgg11_trainer = Trainer(model_name = "VGGNet",size=224)
vgg11_trainer.setup()

CONFONF [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']
CONF [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']


In [ ]:
vgg11_trainer.train()

Epoch 1/10
--------------------


<ipython-input-9-61782fe985c4>:73: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.6785 Acc: 0.5625


<ipython-input-9-61782fe985c4>:105: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.5547 Acc: 0.6630
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.5075 Acc: 0.7450


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.5173 Acc: 0.7717


In [ ]:
vgg11_trainer.test()

<ipython-input-9-61782fe985c4>:127: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

,id,label
0,0,0.191044
1,0,0.778096
2,0,0.070309
3,0,0.197498
4,0,0.745985
5,0,0.894652
6,0,0.090450
7,0,0.155219
8,0,0.455013
9,0,0.299773


# ResNet50

In [9]:
resnet_trainer = Trainer(model_name = "ResNet",size=224)
resnet_trainer.setup()

In [10]:
resnet_trainer.train()

Epoch 1/10
--------------------


<ipython-input-8-8b46cb75c53e>:82: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.train_dataloader):


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7115 Acc: 0.5275


<ipython-input-8-8b46cb75c53e>:113: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs, labels in tqdm(self.val_dataloader):


...

[Val] Loss: 0.6742 Acc: 0.6304
Epoch 10/10
--------------------


  0%|          | 0/13 [00:00<?, ?it/s]

[Train] Loss: 0.7363 Acc: 0.6350


  0%|          | 0/3 [00:00<?, ?it/s]

[Val] Loss: 0.7359 Acc: 0.6087


In [12]:
resnet_trainer.test()

<ipython-input-8-8b46cb75c53e>:135: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for inputs in tqdm(self.test_dataloader):


  0%|          | 0/1 [00:00<?, ?it/s]

,id,label
0,0,0.333761
1,0,0.516337
2,0,0.444796
3,0,0.362532
4,0,0.541789
5,0,0.665701
6,0,0.292702
7,0,0.772925
8,0,0.486886
9,0,0.473149
